In [1]:
import random, json, shutil
import imageio, xmltodict, cv2
from IPython.display import HTML, Video
from tqdm.auto import tqdm
from pathlib import Path
from PIL import Image
from moviepy.editor import ImageSequenceClip

TYPE = '01_Assault'

REL_ROOT = f'../../data1/VOD-unzip/{TYPE}'
sub_dir  = lambda d: [f for f in Path(d).iterdir() if f.is_dir()]
sub_file = lambda d, suffix: [f for f in Path(d).iterdir() if f.suffix==suffix] 
   
dirs = sum([sub_dir(d) for d in sub_dir(REL_ROOT)], [])
xmls = sorted(sum([sub_file(d, '.xml') for d in dirs], []))
mp4s = [f.with_suffix('.mp4')for f in xmls]
len(mp4s)

783

In [2]:
def read_xml(xml):
    with open(xml) as xml_file:
        anno = xmltodict.parse(xml_file.read())['annotation']
    xml_file.close()
    return anno

def last_falldown(xml):
    anno = read_xml(xml)
    objs = anno['object']
    if not isinstance(objs, list):
        objs = [objs]
    
    fdss = []
    for obj in objs:
        actions = obj['action']
        if not isinstance(actions, list):
            actions = [actions]
        fds = [a['frame'] for a in actions if a['actionname'] == 'falldown']
        for idx, fd in enumerate(fds):
            if not isinstance(fd, list):
                fds[idx] = [fd]
        fdss += fds
    fds = sum(fdss, [])
        
    if len(fds) == 0 and 'Assault' in TYPE:
        print('no falldown.', xml)
        return (False, False) 
    else:
        assert 0 < len(fds)
    
    
    last = (0, 0)
    for fd in fds:
        if not isinstance(fd, list):
            fd = [fd]
        last_i = sorted([(int(fd_i['start']), int(fd_i['end'])) for fd_i in fd])[-1]
        if last[1] < last_i[1]:
            last = last_i
    
    return last

def calc_crop_frame(xml):
    s, e = last_falldown(xml)
    if s == False:
        return (False, False)
    s = s + 3*30
    if e < s:
        s = e
    e = random.randint(s, e)
    s = e - 20*30
    e = e + 20*30
    assert 0 < s
    return (s, e) 

def crop_video(mp4, xml):
    s, e = calc_crop_frame(xml)
    if s == False:
        return (False, False)
    cap = cv2.VideoCapture(str(mp4))
    assert cap.isOpened()
    cap.set(cv2.CAP_PROP_POS_FRAMES, s)
    fs = []
    idxs = []
    for i in tqdm(range(s, e)):
        ret, f = cap.read()
        assert ret
        if i%2 == 1:
            continue
        f = cv2.resize(f[:,:,[2,1,0]], (1920, 1080), cv2.INTER_AREA)
        
        fs.append(f)
        idxs.append(i)
        
    return fs, idxs



In [3]:
import os
def resize_and_save(mp4, xml, replace=['VOD-unzip', 'VOD-1920']):
    out_path_mp4 = str(mp4).replace(replace[0], replace[1]) 
    out_path_xml = str(xml).replace(replace[0], replace[1]) 
    if Path(out_path_mp4).exists() and Path(out_path_xml).exists():
        return out_path_mp4
    fs, idxs = crop_video(mp4, xml)
    if fs == False:
        return (False, False)
    p = Path(out_path_mp4).parent
    p.mkdir(parents=True, exist_ok=True)
    if p.exists():
        print('success ')
    else:
        print('fail')
        
    
    # 이미지 복사
    img_dir = Path('/data1/IMG-1920')/mp4.stem
    img_dir.mkdir(exist_ok=True)
    for f, idx in tqdm(zip(fs, idxs), total=len(fs)):
        path = img_dir/f'{mp4.stem}_{idx:05d}.jpg'
        img = Image.fromarray(f)
        img.save(path)
    
    # 동영상 복사
    clip = ImageSequenceClip(fs, 15)
    clip.write_videofile(str(out_path_mp4))
    clip.close()
    
    # xml 복사
    shutil.copy(xml, out_path_xml)
    

In [ ]:
JOB_ID = 0
%pdb on
total = len(mp4s)
mp4s = reversed(mp4s)
xmls = reversed(xmls)

errored = []
for i, (mp4, xml) in enumerate(tqdm(zip(mp4s, xmls), total=total)):
    if i % 1 == JOB_ID:
        try:
            resize_and_save(mp4, xml)
        except:
            errored.append(mp4)

Automatic pdb calling has been turned ON



success 


t:   1%|          | 5/601 [00:00<00:13, 44.71it/s, now=None]


Moviepy - Building video ../../data1/VOD-1920/01_Assault/outsidedoor_08/7-2/7-2_cam02_assault01_place04_day_summer.mp4.
Moviepy - Writing video ../../data1/VOD-1920/01_Assault/outsidedoor_08/7-2/7-2_cam02_assault01_place04_day_summer.mp4



Moviepy - Done !
Moviepy - video ready ../../data1/VOD-1920/01_Assault/outsidedoor_08/7-2/7-2_cam02_assault01_place04_day_summer.mp4


...


Moviepy - Building video ../../data1/VOD-1920/01_Assault/insidedoor_03/24-2/24-2_cam01_assault01_place09_night_summer.mp4.
Moviepy - Writing video ../../data1/VOD-1920/01_Assault/insidedoor_03/24-2/24-2_cam01_assault01_place09_night_summer.mp4



Moviepy - Done !
Moviepy - video ready ../../data1/VOD-1920/01_Assault/insidedoor_03/24-2/24-2_cam01_assault01_place09_night_summer.mp4



success 



Moviepy - Building video ../../data1/VOD-1920/01_Assault/insidedoor_03/24-2/24-2_cam01_assault01_place09_night_spring.mp4.
Moviepy - Writing video ../../data1/VOD-1920/01_Assault/insidedoor_03/24-2/24-2_cam01_assault01_place09_night_spring.mp4



Moviepy - Done !
Moviepy - video ready ../../data1/VOD-1920/01_Assault/insidedoor_03/24-2/24-2_cam01_assault01_place09_night_spring.mp4


In [ ]:
errored